In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# 1. Charger le fichier
df = pd.read_csv("C:/Users/HP G3/Downloads/cars_avito.csv")

# 2. Nettoyage de la colonne 'Kilométrage'
def convert_km_range_to_median(val):
    if isinstance(val, str) and '-' in val:
        parts = val.split('-')
        try:
            low = int(parts[0].replace(' ', '').strip())
            high = int(parts[1].replace(' ', '').strip())
            return int((low + high) / 2)
        except:
            return np.nan
    try:
        return int(val.replace(' ', '').strip())
    except:
        return np.nan

df['Kilométrage'] = df['Kilométrage'].apply(convert_km_range_to_median)

# 3. Identifier les colonnes numériques et non numériques
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# 4. Imputation prédictive pour les colonnes numériques
imputer = IterativeImputer(estimator=RandomForestRegressor(), random_state=0)
df_numeric = pd.DataFrame(imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)

# Forcer les prédictions à des entiers
df_numeric = df_numeric.round(0).astype(int)

# 6. Remplir les valeurs manquantes des colonnes non numériques avec leur mode
df_non_numeric = df[non_numeric_cols].copy()
for col in df_non_numeric.columns:
    if df_non_numeric[col].isna().sum() > 0:
        mode = df_non_numeric[col].mode()
        if not mode.empty:
            df_non_numeric[col] = df_non_numeric[col].fillna(mode[0])

# 6. Fusion
df_cleaned = pd.concat([df_numeric, df_non_numeric], axis=1)

# 7. Sauvegarde
df_cleaned.to_csv("cars_avito_cleaned.csv", index=False)
print("✅ Fichier nettoyé et sauvegardé sous le nom 'cars_avito_cleaned.csv'")

# 8. Vérification
missing = df_cleaned.isnull().sum()
if missing.sum() == 0:
    print("✅ Toutes les colonnes sont remplies, aucune valeur manquante détectée.")
else:
    print("⚠️ Il reste des valeurs manquantes dans les colonnes suivantes :")
    print(missing[missing > 0])


✅ Fichier nettoyé et sauvegardé sous le nom 'cars_avito_cleaned.csv'
✅ Toutes les colonnes sont remplies, aucune valeur manquante détectée.


In [4]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("cars_avito_cleaned.csv")

# Calculer le mode parmi 'Manuelle' et 'Automatique'
mode_bv = df[df['BoiteàV'].isin(['Manuelle', 'Automatique'])]['BoiteàV'].mode()[0]

# Remplacer les valeurs différentes de 'Manuelle' ou 'Automatique' par le mode
df['BoiteàV'] = df['BoiteàV'].apply(lambda x: x if x in ['Manuelle', 'Automatique'] else mode_bv)

# Sauvegarder le résultat dans un nouveau fichier
df.to_csv("cars_avitoo_cleaned.csv", index=False)
print("✅ Fichier sauvegardé sous le nom 'cars_avitoo_cleaned.csv'")


✅ Fichier sauvegardé sous le nom 'cars_avitoo_cleaned.csv'


In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html

# Charger les données nettoyées
df = pd.read_csv("cars_avito_cleaned.csv")

# 1. Préparation des données
# Remplacer les valeurs incorrectes dans la colonne 'Année'
def convert_year(val):
    try:
        return int(val)
    except ValueError:
        return 1980  # Valeur par défaut

df['Année'] = df['Année'].apply(convert_year)

# Calculer l'âge des voitures
df['Age'] = 2025 - df['Année']

# 2. Division du prix par 100 pour une meilleure visualisation
df['Prix'] = df['Prix'] / 100

# 3. Suppression des valeurs aberrantes pour le prix et l'âge
q1 = df['Prix'].quantile(0.25)
q3 = df['Prix'].quantile(0.75)
IQR = q3 - q1
df = df[(df['Prix'] >= q1 - 1.5 * IQR) & (df['Prix'] <= q3 + 1.5 * IQR)]

# 4. Visualisation des histogrammes

# 4.1 Histogramme de la distribution des prix
fig_price = px.histogram(df, x='Prix', nbins=30, title='Distribution des Prix des Voitures',
                          labels={'Prix': 'Prix en Dirhams'}, template="plotly_dark", color_discrete_sequence=["#FF6347"])

# 4.2 Histogramme de la répartition des prix par marque
fig_boxplot = px.histogram(df, x='Marque', y='Prix', title="Répartition des Prix par Marque",
                            template="plotly_dark", color_discrete_sequence=["#8A2BE2"])

# 4.3 Histogramme de la répartition des prix par type de carburant
fig_carburant = px.histogram(df, x='Carburant', y='Prix', title="Répartition des Prix par Type de Carburant",
                              template="plotly_dark", color_discrete_sequence=["#3CB371"])

# 4.4 Histogramme de l'impact de l'âge du véhicule sur le prix
fig_age = px.histogram(df, x='Age', y='Prix', title="Impact de l'Âge du Véhicule sur le Prix",
                        template="plotly_dark", color_discrete_sequence=["#FF1493"])

# 5. Calculer la matrice de corrélation sur les données numériques uniquement
df_numeric = df.select_dtypes(include=['float64', 'int64'])
corr_matrix = df_numeric.corr()

# 6. Heatmap des corrélations entre les variables
fig_heatmap = px.imshow(corr_matrix, text_auto=True, aspect="auto",
                        title="Heatmap des Corrélations entre les Variables", color_continuous_scale="YlGnBu")

# 7. Visualisation des modèles les plus populaires
top_models = df['Modèle'].value_counts().head(10)
fig_models = px.bar(top_models, x=top_models.index, y=top_models.values,
                    title="Top 10 des Modèles les Plus Populaires",
                    labels={'x': 'Modèle', 'y': 'Nombre d\'occurrences'}, template="plotly_dark", color_discrete_sequence=["#FFD700"])

# 8. Répartition des véhicules selon l'origine
fig_origine = px.pie(df, names='Origine', title="Répartition des Véhicules par Origine",
                     template="plotly_dark", color_discrete_sequence=["#FF8C00", "#DAA520"])


# 11. Répartition des voitures par type de boîte de vitesses (Automatique et Manuelle)
df_boite_vitesse = df[df['BoiteàV'].isin(['Automatique', 'Manuelle'])]  # Filtrer uniquement les types 'Automatique' et 'Manuelle'

fig_boite_vitesse = px.pie(df_boite_vitesse, names='BoiteàV', title="Répartition des Voitures par Type de Boîte à Vitesses",
                            template="plotly_dark", color_discrete_sequence=["#FF6347", "#4682B4"])

# 14. Répartition des véhicules par état
fig_etat = px.pie(df, names='État', title="Répartition des Voitures par État",
                  template="plotly_dark", color_discrete_sequence=["#DAA520", "#8B4513", "#D2691E"])

# 15. Visualisation des données avec Dash
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Analyse Exploratoire des Données - Marché Automobile Marocain', style={'textAlign': 'center', 'color': '#FF6347'}),
    html.Div([
        dcc.Graph(figure=fig_price),
        dcc.Graph(figure=fig_boxplot),
        dcc.Graph(figure=fig_carburant),
        dcc.Graph(figure=fig_age),
        dcc.Graph(figure=fig_heatmap),
        dcc.Graph(figure=fig_models),
        dcc.Graph(figure=fig_origine),
        dcc.Graph(figure=fig_boite_vitesse),
        dcc.Graph(figure=fig_etat)
    ], style={'display': 'grid', 'gridTemplateColumns': 'repeat(2, 1fr)', 'gap': '20px'})
])

if __name__ == '__main__':
    app.run(debug=True, port=8056)
